In [1]:
## LOADING THE REQUIRED LIBRARIES

import pandas as pd
import numpy as np
import os
import re
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
warnings.filterwarnings('ignore')
from tqdm import tqdm 
import tensorflow as tf
from  tensorflow.keras.preprocessing.sequence import pad_sequences
from  sklearn.model_selection import train_test_split
from tqdm import tqdm

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
from tensorflow.python.client import device_lib

Num GPUs Available:  0


2022-10-17 22:15:43.478946: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2022-10-17 22:15:43.478977: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# The GPU id to use, "0" to  "7" 
os.environ["CUDA_VISIBLE_DEVICES"]="2,3,4,6,7"

In [3]:
# Loading Dataset
## LOADING THE PROCESSED DATASET
df= pd.read_csv("DATA/processed_data_lang8.csv")
df.columns = ["enc_input","dec_input"] 
df["dec_output"] = df.dec_input
df

,enc_input,dec_input,dec_output
0,and he took in my favorite subject like soccer,and he took in my favorite subjects like soccer,and he took in my favorite subjects like soccer
1,actually who let me know about lang 8 was him,actually he was the one who let me know about ...,actually he was the one who let me know about ...
2,his kanji is ability is much better than me,his kanji ability is much better than mine,his kanji ability is much better than mine
3,we have known each other for only half a year ...,we have known each other for only half a year ...,we have known each other for only half a year ...
4,i heard a sentence last night when i watched tv,i heard a sentence last night when i was watch...,i heard a sentence last night when i was watch...
...,...,...,...
503896,i like thailand language because that pronounc...,i like thai because the pronunciation sounds cute,i like thai because the pronunciation sounds cute
503897,i ate kaomangai rise with boild chikin tomya...,i ate kaomangai rice with boiled chickin tom...,i ate kaomangai rice with boiled chickin tom...
503898,i think it is important thing to become to lik...,i think it is important to like coriander in o...,i think it is important to like coriander in o...
503899,yesterday i went to umeda station to date,i went to umeda station for dating yesterday,i went to umeda station for dating yesterday


In [4]:
## THE INPUTS TO THE DECODER REQUIRES SPECIAL TOKENS FOR THE START AND THE END SO WE ARE GOING TO USE 
## <start> AS BEGINING TOKEN
## <end>  AS END TOKEN

df["dec_input"]= "<start> " + df["dec_input"]
df["dec_output"] =  df["dec_output"] + " <end>" 
df

,enc_input,dec_input,dec_output
0,and he took in my favorite subject like soccer,<start> and he took in my favorite subjects li...,and he took in my favorite subjects like socce...
1,actually who let me know about lang 8 was him,<start> actually he was the one who let me kno...,actually he was the one who let me know about ...
2,his kanji is ability is much better than me,<start> his kanji ability is much better than ...,his kanji ability is much better than mine <end>
3,we have known each other for only half a year ...,<start> we have known each other for only half...,we have known each other for only half a year ...
4,i heard a sentence last night when i watched tv,<start> i heard a sentence last night when i w...,i heard a sentence last night when i was watch...
...,...,...,...
503896,i like thailand language because that pronounc...,<start> i like thai because the pronunciation ...,i like thai because the pronunciation sounds c...
503897,i ate kaomangai rise with boild chikin tomya...,<start> i ate kaomangai rice with boiled chic...,i ate kaomangai rice with boiled chickin tom...
503898,i think it is important thing to become to lik...,<start> i think it is important to like corian...,i think it is important to like coriander in o...
503899,yesterday i went to umeda station to date,<start> i went to umeda station for dating yes...,i went to umeda station for dating yesterday <...


In [5]:
# Splitting And Sampling around 100k datapoints
#THE TOTAL DATASET HAS 500K DATAPOINTS WHICH WILL TAKE MUCH HIGHER TRAINING TIME. THEREFORE I AM SAMPLING ONE-FIFTH OF THE TOTAL DATASET

#df_sampled = pd.concat((df[df.enc_input].sample(frac= 0.2,random_state=1)))
df_sampled = df.sample(frac = 0.2)
print(df.shape)
print(df_sampled.shape)

(503901, 3)
(100780, 3)


In [6]:
## ONCE THE DATA IS SAMPLED WE ARE SPLITTIND THE DATA IN TO TRAIN AND TEST
df_train ,df_val = train_test_split(df_sampled,test_size=0.2,random_state = 3)

In [7]:
## IN THE COLUMN WHICH HAS DECODER INPUTS ADDING "<end>" TOKEN TO BE LEARNED BY THE TOKENIZER
df_train["dec_input"].iloc[0]  = df_train.iloc[0]["dec_input"] + " <end>"
df_train

,enc_input,dec_input,dec_output
298469,i try to write english,<start> i try to write in english <end>,i try to write in english <end>
431393,we could not speak korea neither,<start> we could not speak korean either,we could not speak korean either <end>
193781,i wonder if book stores will be able to surviv...,<start> i wonder nbsp how book stores will b...,i wonder nbsp how book stores will be able t...
13435,i wonder how much it cost to buy furniture ele...,<start> i wonder how much it will cost to buy ...,i wonder how much it will cost to buy furnitur...
403221,the cost is covers by insurance company,<start> the cost is covered by an insurance co...,the cost is covered by an insurance company <end>
...,...,...,...
241570,today i was twenty minutes late to my first cl...,<start> today i was twenty minutes late to fir...,today i was twenty minutes late to first perio...
306559,otherwise i would never be able to help my cus...,<start> otherwise i would never be able to hel...,otherwise i would never be able to help my cus...
401908,if he she is fascinated by the amazing scener...,<start> if one is fascinated by the amazing sc...,if one is fascinated by the amazing scenery of...
152383,after drinking i saw some unbelievable young g...,<start> after drinking i saw some unbelievable...,after drinking i saw some unbelievable young g...


In [8]:
## VALIDATION DATA
df_val

,enc_input,dec_input,dec_output
357371,eventually i returned to my usual pc that runs xp,<start> eventually i returned to my usual comp...,eventually i returned to my usual computer whi...
416692,i contrived to arrive in time after all but my...,<start> i managed to arrive in time after all ...,i managed to arrive in time after all but my k...
218309,so i am in office now because i have to transl...,<start> so i am in office now because i have t...,so i am in office now because i have to transl...
385646,whether or not we can win the tournament this ...,<start> whether or not we can win the tourname...,whether or not we can win the tournament this ...
314161,but ordinary one is sometimes very delicious,<start> but sometimes ordinary ones are very d...,but sometimes ordinary ones are very delicious...
...,...,...,...
157003,instead of that the entrance fee is reasonable...,<start> instead the entrance fee is reasonable...,instead the entrance fee is reasonable and the...
35566,when i was college student i worked at coffee ...,<start> when i was college student i worked at...,when i was college student i worked at a coffe...
36390,it is so happy for me because it is a chance t...,<start> it is such a happy time for me becasus...,it is such a happy time for me becasuse it is ...
224812,i participate in the examination called sia,<start> i will participate in the examination ...,i will participate in the examination called s...


In [9]:
## HERE I AM SAMPLING 1000 POINTS FROM THE DATAFRAME AS TEST DATA WHICH ARE NOT PRESEENT IN THE TRAIN AND VALIDAION DATA
np.random.seed(5) 
df_test = df.loc[np.random.choice(np.array([x for x in df.index.values if x not in df_sampled.index.values]),1000,replace= False,)]
df_test

,enc_input,dec_input,dec_output
144764,na ka is the politely final particle in thai ...,<start> na ka is the politely final particle ...,na ka is the politely final particle in thai ...
455909,even thought i know i should do all my best to...,<start> even though i know i should do all my ...,even though i know i should do all my best to ...
13948,a dialogue in a bus,<start> a dialogue on a bus,a dialogue on a bus <end>
129170,as a result this wise emperor not only worked ...,<start> as a result this wise emperor not only...,as a result this wise emperor not only worked ...
402708,i do not like the exam of pediatrics for it is...,<start> i do not like the exam for pediatrics ...,i do not like the exam for pediatrics it is di...
...,...,...,...
352610,it is about going to the poor country as a vol...,<start> it is about going to a poor countrysid...,it is about going to a poor countryside as a v...
191594,we can only speak english there and there are ...,<start> while there we are only allowed to spe...,while there we are only allowed to speak engli...
179915,whenever i have a time i will visit there to s...,<start> whenever i have the time i will visit ...,whenever i have the time i will visit there to...
38631,what is more teachers can change the way of in...,<start> what is more teachers can change their...,what is more teachers can change their way of ...


In [10]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [11]:
## TOKENIZER FOR ENCODER INPUT
tk_inp = Tokenizer()
tk_inp.fit_on_texts(df_train.enc_input.apply(str))

In [12]:
# TOKENIZER FOR DECODER INPUT
tk_out = Tokenizer(filters='!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n' )
tk_out.fit_on_texts(df_train.dec_input.apply(str))

In [13]:
## THIS CLASS CONVERTS TEXT DATA TO INTEGER SEQUENCES AND RETURNS THE PADDED SEQUENCES
class Dataset :
    def __init__(self, data , tk_inp ,tk_out, max_len):
        ## SETTING THE REQUIRED ATTRIBUTES
        self.encoder_inp = data["enc_input"].apply(str).values
        self.decoder_inp = data["dec_input"].apply(str).values
        self.decoder_out = data["dec_output"].apply(str).values
        self.tk_inp = tk_inp
        self.tk_out = tk_out
        self.max_len = max_len
        
    def __getitem__(self,i):
        # INPUT SEQUENCES
        self.encoder_seq = self.tk_inp.texts_to_sequences([self.encoder_inp[i]])
        # DECODER INPUT SEQUENCES 
        self.decoder_inp_seq = self.tk_out.texts_to_sequences([self.decoder_inp[i]])
        # DECODER INPUT SEQUENCES
        self.decoder_out_seq = self.tk_out.texts_to_sequences([self.decoder_out[i]])
        
        # PADDING THE ENCODER INPUT SEQUENCES
        self.encoder_seq = pad_sequences(self.encoder_seq, padding="post",maxlen = self.max_len)
        # PADDING THE DECODER INPUT SEQUENCES
        self.decoder_inp_seq = pad_sequences(self.decoder_inp_seq, padding="post",maxlen = self.max_len)
        # PADDING DECODER OUTPUT SEQUENCES
        self.decoder_out_seq = pad_sequences(self.decoder_out_seq ,padding="post", maxlen = self.max_len)
        return self.encoder_seq ,  self.decoder_inp_seq,  self.decoder_out_seq
    
    def __len__(self):
        # RETURN THE LEN OF INPUT ENDODER
        return len(self.encoder_inp)

In [14]:
## THIS CLASS CONVERTES THE DATASET INTO THE REQUIRED BATCH SIZE

class Dataloader(tf.keras.utils.Sequence):
    def __init__(self,batch_size,dataset):
        # INTIALIZING THE REQUIRED VARIABLES 
        self.dataset = dataset
        self.batch_size = batch_size
        self.totl_points = self.dataset.encoder_inp.shape[0]
        
    def __getitem__(self,i):
        # STATING THE START AND STOP VATIABLE CONTAINGING INDEX VALUES FOR EACH BATCH
        start = i * self.batch_size
        stop = (i+1)*self.batch_size
        
        # PLACEHOLDERS FOR BATCHED DATA
        batch_enc =[]
        batch_dec_input = []
        batch_dec_out =[]

        for j in range(start,stop): 
            
            a,b,c = self.dataset[j] 
            batch_enc.append(a[0]) 
            batch_dec_input.append(b[0])
            batch_dec_out.append(c[0]) 
        
        # Conveting list to array   
        batch_enc = (np.array(batch_enc)) 
        batch_dec_input = np.array(batch_dec_input)
        batch_dec_out = np.array(batch_dec_out)
        
        return [batch_enc , batch_dec_input],batch_dec_out
    
    def __len__(self):
        # Returning the number of batches
        return int(self.totl_points/self.batch_size)

In [15]:
# FORMING OBJECTS OF DATASET AND DATALOADER FOR TRAIN DATASET
train_dataset = Dataset(df_train,tk_inp,tk_out,35)
train_dataloader = Dataloader( batch_size = 512, dataset=train_dataset)

# FORMING OBJECTS OF DATASET AND DATALOADER FOR VALIDATION DATASET
val_dataset = Dataset(df_val , tk_inp,tk_out,35)
val_dataloader = Dataloader(batch_size=512 , dataset=val_dataset)

In [16]:
#Attention Model
from tensorflow.keras import layers
from tensorflow.keras import Model

In [17]:
## DEFINING THE ENCODER LAYER AS A FUNCTION

class Encoder(tf.keras.layers.Layer):


    
    def __init__(self, vocab_size,emb_dims, enc_units, input_length,batch_size):
        super().__init__()
        # INITIALIZING THE REQUIRED VARIABLES
        self.batch_size=batch_size # BATHCH SIZE
        self.enc_units = enc_units # ENCODER UNITS

        # EMBEDDING LAYER
        self.embedding= layers.Embedding(vocab_size ,emb_dims) 
        # LSTM LAYER WITH RETURN SEQ AND RETURN STATES
        self.lstm = layers.LSTM(self.enc_units,return_state= True,return_sequences =  True) 
    def call(self, enc_input , states):
      
        # FORMING THE EMBEDDED VECTOR 
        emb = self.embedding(enc_input)
        # PASSING THE EMBEDDED VECTIO THROUGH LSTM LAYERS 
        enc_output,state_h,state_c = self.lstm(emb,initial_state=states)
        #RETURNING THE OUTPUT OF LSTM LAYER
        return enc_output,state_h,state_c 
    def initialize(self,batch_size):

        return tf.zeros(shape=(batch_size,self.enc_units)),tf.zeros(shape=(batch_size,self.enc_units))

In [18]:
# THIS IS ATTNETION LAYER FOR DOT MODEL
class Attention(tf.keras.layers.Layer):
    
    def __init__(self,units):
        super().__init__()
        # INITIALIZING THE DENSE LAYER W1
        self.W1 = layers.Dense(units)
        # INITIALIZING THE DENSE LAYER W2
        self.W2 = layers.Dense(units)
        # INITIALIZING THE DENSE LAYER V
        self.v = layers.Dense(1)
        
    def call(self,enc_output,dec_state):
        # EXPANDING THE DIMENSION OF DECODER STATE  EG. FROM (16,32) TO (16,32,1)
        dec_state =  tf.expand_dims(dec_state,axis=1)
        
        # FINDING THE SCORE FOR CONCAT MODEL
        score = self.v(tf.nn.tanh(
            self.W1(dec_state)+ self.W2(enc_output)
        ))
        # APPLYING SOFTMAX TO THE AXIS 1
        # OUPUT SHAPE = (16,13,1)
        att_weights = tf.nn.softmax(score,axis=1)
        
        # CALCULATING THE CONTEXT VECTOR BY FIRST ELEMENTWISE MULTIPLICATION AND THEN ADDING THE AXIS 1
        # (16,13,1)*(16,13,32)=(16,13,32)
        context_vec  = att_weights* enc_output
        
        # (16,13,32) SUM AND REDUCE THE DIMENSION AT AXIS 1 => (16,32)
        context_vec = tf.reduce_sum(context_vec,axis=1)
        
        # RETURNING THE CONTEXT VECTOR AND ATTENTION WEIGHTS
        return context_vec,att_weights

In [19]:
class Onestepdecoder(tf.keras.Model):
    '''THIS MODEL OUTPUTS THE RESULT OF DECODER FOR ONE TIME SETP GIVEN THE INPUT FOR PRECIOVE TIME STEP'''
    
    def __init__(self, vocab_size,emb_dims, dec_units, input_len,att_units,batch_size):
        super().__init__()
        # INTITALIZING THE REQUIRED VARIABLES
        # EMBEDDING LAYERS
        self.emb = layers.Embedding(vocab_size,emb_dims,input_length= input_len)
        # ATTENTION LAYER
        self.att = Attention(att_units)
        # LSTM LAYER
        self.lstm = layers.LSTM(dec_units,return_sequences=True,return_state=True)
        # DENSE LAYER
        self.dense = layers.Dense(vocab_size,activation="softmax")

    def call(self, encoder_output , input , state_h,state_c):
        # FORMING THE EMBEDDED VECTOR FOR THE WORD
        # (32,1)=>(32,1,12)
        emb = self.emb(input)

        dec_output,dec_state_h,dec_state_c = self.lstm(emb, initial_state = [state_h,state_c] )

        # GETTING THE CONTEXT VECTOR AND ATTENTION WEIGHTS BASED ON THE ENCODER OUTPUT AND  DECODER STATE_H
        context_vec,alphas = self.att(encoder_output,dec_state_h)
        
        # CONCATINATING THE CONTEXT VECTOR(BY EXPANDING DIMENSION) AND ENBEDDED VECTOR
        dense_input =  tf.concat([tf.expand_dims(context_vec,1),dec_output],axis=-1)
        
        # PASSING THE DECODER OUTPUT THROUGH DENSE LAYER WITH UNITS EQUAL TO VOCAB SIZE
        fc = self.dense(dense_input)
        
        # RETURNING THE OUTPUT
        return fc , dec_state_h , dec_state_c , alphas

In [20]:
class Decoder(tf.keras.Model):
    '''THIS MODEL PERFORMS THE WHOLE DECODER OPERATION FOR THE COMPLETE SENTENCE'''
    def __init__(self, vocab_size,emb_dims, dec_units, input_len,att_units,batch_size):
        super().__init__()
        # INITIALIZING THE VARIABLES
        # LENGTH OF INPUT SENTENCE
        self.input_len = input_len
        # ONE STEP DECODER
        self.onestepdecoder = Onestepdecoder(vocab_size,emb_dims, dec_units, input_len,att_units,batch_size)

    def call(self,dec_input,enc_output,state_h,state_c):
        # THIS VATIABLE STORES THE VALUE OF STATE_H FOR THE PREVIOUS STATE
        current_state_h = state_h 
        current_state_c = state_c
        # THIS STORES THE DECODER OUTPUT FOR EACH TIME STEP
        pred = []
        # THIS STORED THE ALPHA VALUES
        alpha_values = []
        # FOR EACH WORD IN THE INPUT SENTENCE
        for i in range(self.input_len):
            
            # CURRENT WORD TO INPUT TO ONE STEP DECODER
            current_vec = dec_input[:,i]

            # EXPANDING THE DIMENSION FOR THE WORD
            current_vec = tf.expand_dims(current_vec,axis=-1)

            # PERFORMING THE ONE STEP DECODER OPERATION 
            dec_output,dec_state_h,dec_state_c,alphas = self.onestepdecoder(enc_output ,current_vec,current_state_h,current_state_c)

            #UPDATING THE CURRENT STATE_H
            current_state_h = dec_state_h
            current_state_c = dec_state_c

            #APPENDING THE DECODER OUTPUT TO "pred" LIST
            pred.append(dec_output)

            # APPENDING THE ALPHA VALUES
            alpha_values.append(alphas)
            
        # CONCATINATING ALL THE VALUES IN THE LIST
        output = tf.concat(pred,axis=1)
        # CONCATINATING ALL THE ALPHA VALUES IN THE LIST
        alpha_values = tf.concat(alpha_values,axis = -1)
        # RETURNING THE OUTPUT
        return output , alpha_values

In [21]:
class encoder_decoder(tf.keras.Model):
    '''THIS MODEL COMBINES ALL THE LAYERS AND FORM IN ENCODER DECODER MODEL WITH ATTENTION MECHANISM'''
    def __init__(self,enc_vocab_size,enc_emb_dim,enc_units,enc_input_length,
             dec_vocab_size,dec_emb_dim,dec_units,dec_input_length ,att_units, batch_size):
        # INITAILIZING ALL VARIABLES
        super().__init__()
        # BATCH SIZE
        self.batch_size = batch_size
        # INITIALIZING ENCODER LAYER
        self.encoder = Encoder(enc_vocab_size, enc_emb_dim,enc_units, enc_input_length,batch_size)
        # INITALIZING DECODER LAYER
        self.decoder = Decoder(dec_vocab_size ,dec_emb_dim,dec_units,dec_input_length  ,att_units, batch_size)

    def call(self,data):
        # THE INPUT OF DATALOADER IS IN A LIST FORM FOR EACH BATCH IT GIVER TWO INPUTS
        # INPUT1 IS FOR ENCODER
        # INPUT2 IS FOR DECODER
        inp1 , inp2 = data
        # PASSING THE INPUT1 TO ENCODER LAYER
        enc_output, enc_state_h, enc_state_c = self.encoder(inp1,self.encoder.initialize(self.batch_size))
        # PASSING INPUT2 TO THE DECODER LAYER
        dec_output , alphas = self.decoder(inp2 , enc_output,enc_state_h,enc_state_c)
        # THE OUTPUT OF MODEL IS ONLY DECODER OUTPUT THE ALPHA VALUES ARE IGNORED HERE
        return dec_output

In [22]:
# INITAILZING THE MODEL
model = encoder_decoder(enc_vocab_size=len(tk_inp.word_index)+1,
                         enc_emb_dim = 300,
                         enc_units=256,enc_input_length=35,
                         dec_vocab_size =len(tk_out.word_index)+1,
                         dec_emb_dim =300,
                         dec_units=256,
                         dec_input_length = 35,
                         
                         att_units=256,
                         batch_size=512)

2022-10-17 22:16:13.902485: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [23]:
callback =[ tf.keras.callbacks.ModelCheckpoint( "model_save/attention_concat_best.h5",save_best_only=True,mode="min" ,save_weights_only=True),
           tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=5,verbose=1,min_delta=0.0001),
            tf.keras.callbacks.TensorBoard("model_save/attention_concat_logs_save",histogram_freq=1)
]

train_steps = train_dataloader.__len__()
val_steps  = val_dataloader.__len__()

model.compile(optimizer="adam",loss='sparse_categorical_crossentropy')

In [24]:
model.fit(train_dataloader, steps_per_epoch=train_steps,epochs= 50,validation_data = val_dataloader,validation_steps =val_steps,callbacks=callback)

Epoch 1/50
157/157 [==============================] - 769s 5s/step - loss: 2.9408 - val_loss: 2.1825
Epoch 2/50
157/157 [==============================] - 683s 4s/step - loss: 2.1631 - val_loss: 2.0018
Epoch 3/50
157/157 [==============================] - 684s 4s/step - loss: 1.9732 - val_loss: 1.8471
Epoch 4/50
157/157 [==============================] - 681s 4s/step - loss: 1.8514 - val_loss: 1.7665
Epoch 5/50
157/157 [==============================] - 679s 4s/step - loss: 1.7676 - val_loss: 1.7069
Epoch 6/50
157/157 [==============================] - 677s 4s/step - loss: 1.6901 - val_loss: 1.6419
Epoch 7/50
157/157 [==============================] - 678s 4s/step - loss: 1.6016 - val_loss: 1.5506
Epoch 8/50
157/157 [==============================] - 677s 4s/step - loss: 1.4665 - val_loss: 1.4015
Epoch 9/50
157/157 [==============================] - 675s 4s/step - loss: 1.2531 - val_loss: 1.1556
Epoch 10/50
157/157 [==============================] - 678s 4s/step - loss: 0.9809 - val_lo

In [25]:
model.build([(512,35),(512,35)])
model.summary()

Model: "encoder_decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder (Encoder)           multiple                  10050368  
                                                                 
 decoder (Decoder)           multiple                  22305245  
                                                                 
Total params: 32,355,613
Trainable params: 32,355,613
Non-trainable params: 0
_________________________________________________________________


In [26]:
model.load_weights("model_save/attention_concat_best.h5")

In [27]:
def predict(ita_text,model):
    '''THIS FUNCTION IS USED IN INFERENCE TIME WHICH GIVEN ANY SENTENCE IN ITALIAN OUTPUTS THE ENGLISH SENTENCE AND ALPHA VALUES'''
    # FORMING TOKENIZED SEQUENCES FOR INPUT SENTENCE
    seq = tk_inp.texts_to_sequences([ita_text])
    # PADDING THE SEQUENCES
    seq = pad_sequences(seq,maxlen = 20 , padding="post")
    # INITIALIZING THE STATES FOR INPUTING TO ENCODER
    state = model.layers[0].initialize(1)
    # GETTING THE ENCODED OUTPUT
    enc_output,state_h,state_c= model.layers[0](seq,state)
    # VARIABLE TO STORE PREDICTED SENTENCE
    pred = []
    # THIS VARIABLE STORES THE STATE TO BE INPUTED TO ONE STEP ENCODER
    input_state_h = state_h
    input_state_c = state_c
    # THIS VARIABLE STORES THE VECTOR TO VE INPUTED TO ONE STEP ENCODER
    current_vec = tf.ones((1,1))
    # THIS VARIABLE WILL STORE ALL THE ALPHA VALUES OUTPUTS
    alpha_values = []

    for i in range(20):
        # PASSING THE REQUIRED VARIABLE TO ONE STEP ENCODER LAYER
        fc , dec_state_h ,dec_state_c, alphas = model.layers[1].layers[0](enc_output , current_vec ,input_state_h ,input_state_c)
        #APPENDING THE ALPHA VALUES TO THE LIST "alpha_values"
        alpha_values.append(alphas)
         # UPDATING THE CURRENT VECTOR 
        current_vec = np.argmax(fc , axis = -1)
         # UPDATING THE INPUT STATE
        input_state_h = dec_state_h
        input_state_c = dec_state_c
        # GETTING THE ACTUAL WORDS FRO THE TOKENIZED INDEXES
        pred.append(tk_out.index_word[current_vec[0][0]])
        # IF THE WORD "<end>" COMES THE LOOP WILL BREAK
        if tk_out.index_word[current_vec[0][0]]=="<end>":
              break
    # JOINING THE PREDICTED WORDS
    pred_sent = " ".join(pred)
    # CONCATINATING ALL THE ALPHA VALUES
    alpha_values = tf.squeeze(tf.concat(alpha_values,axis=-1),axis=0)
    # RETURNING THE PREDICTED SENTENCE AND ALPHA VALUES
    return  pred_sent , alpha_values

In [28]:
import nltk.translate.bleu_score as bleu

In [29]:
BLEU = []
index = []
np.random.seed(1)
test_data = df_val.loc[np.random.choice(df_val.index,size = 2000,replace=False)]
for ind,i in tqdm(test_data.iterrows(),position=0):
    try:
        pred = predict(str(i.enc_input),model)[0].split()
        act = [str(i.dec_output).split()]
        b = bleu.sentence_bleu(act,pred)
        BLEU.append(b)
    except:
        index.append(ind)
        continue
print("BELU = ", np.mean(BLEU))

2000it [07:16,  4.58it/s]

BELU =  0.4035954595719226


In [30]:
print("INPUT SENTENCE ===> ",df_test.enc_input.values[19])
print("PREDICTED SENTENCE ===> ",predict(df_test.enc_input.values[19],model)[0])
print("ACTUAL SENTENCE ===> ",df_test.dec_output.values[19])

INPUT SENTENCE ===>  he went to nagano in school trip and ski there
PREDICTED SENTENCE ===>  he went to nagano in school trip and the ski festival <end>
ACTUAL SENTENCE ===>  he went to nagano once on a school trip and skied there <end>


In [31]:
print("INPUT SENTENCE ===> ",df_test.enc_input.values[50])
print("PREDICTED SENTENCE ===> ",predict(df_test.enc_input.values[50],model)[0])
print("ACTUAL SENTENCE ===> ",df_test.dec_output.values[50])

INPUT SENTENCE ===>  yesterday i checked how to set the environment up for developing iphone applications using other is account
PREDICTED SENTENCE ===>  yesterday i checked how to set the environment up for developing the iphone applications using other is account <end>
ACTUAL SENTENCE ===>  yesterday i checked how to set the environment up for developing iphone applications using another person is account <end>


In [32]:
%%time
predict(df_test.enc_input.values[50],model)[0]

CPU times: user 340 ms, sys: 0 ns, total: 340 ms
Wall time: 336 ms


'yesterday i checked how to set the environment up for developing the iphone applications using other is account <end>'

In [33]:
def beam_search(input,model,k):
    seq = tk_inp.texts_to_sequences([input])
    seq = pad_sequences(seq,maxlen = 35,padding="post")

    state = model.layers[0].initialize(1)
    # GETTING THE ENCODED OUTPUT
    enc_output,enc_state_h,enc_state_c = model.layers[0](seq,state)
    

    input_state_h = enc_state_h
    input_state_c = enc_state_c 
    k_beams = [[tf.ones((1,1),dtype=tf.int32),0.0]]
    for i in range(35):
        candidates = []
        for sent_pred , prob in k_beams :
            if tk_out.word_index["<end>"] in sent_pred.numpy() :
                candidates += [[sent_pred , prob]]
            else:
               
                dec_input = model.layers[1].layers[0].layers[0](sent_pred)
                dec_output , dec_state_h , dec_state_c   =  model.layers[1].layers[0].layers[2](dec_input ,  initial_state =  [input_state_h , input_state_c])

                context_vec , alphas =  model.layers[1].layers[0].layers[1](enc_output,dec_state_h)

                # CONCATINATING THE CONTEXT VECTOR(BY EXPANDING DIMENSION) AND ENBEDDED VECTOR
                dense_input =  tf.concat([tf.expand_dims(context_vec,1),tf.expand_dims(dec_state_h,1)],axis=-1)
                
                # PASSING THE DECODER OUTPUT THROUGH DENSE LAYER WITH UNITS EQUAL TO VOCAB SIZE
                dense = model.layers[1].layers[0].layers[3](dense_input)

                pred = tf.argsort(dense, direction= 'DESCENDING')[:,:,:k]
                for w in range(k):
                  candidates += [[tf.concat((sent_pred, pred[:,:,w]) , axis=-1) , (prob + tf.math.log(dense[:,:,pred[:,:,w][0][0]])[0][0])]  ]
        k_beams = sorted(candidates,key=lambda tup:tup[1],reverse=True)[:k]

    all_sent = []
    for i,score in k_beams:
        sent = ""
        for j in range(1,35):
            sent +=  tk_out.index_word[i.numpy()[:,j][0]] +  " " 
            if tk_out.index_word[i.numpy()[:,j][0]] =="<end>":
                break
        all_sent.append((sent.strip(),score.numpy()))
    return all_sent

In [34]:
# VALIDATION BELU SCORE
BLEU_beam = []
index = []
np.random.seed(1)
test_data = df_val.loc[np.random.choice(df_val.index,size = 2000,replace=False)]
for ind,i in tqdm(test_data.iterrows(),position=0):
    try:
        pred = beam_search(str(i.enc_input),model,3)[0][0].split()
        act = [str(i.dec_output).split()]
        b =bleu.sentence_bleu(act,pred)
        BLEU_beam.append(b)
    except:
        index.append(ind)
        continue

print("BELU Score = ",np.mean(BLEU_beam)) 

2000it [36:44,  1.10s/it]

BELU Score =  0.442634669702357


In [35]:
print("INPUT SENTENCE ===> ",df_test.enc_input.values[19])
print("="*50)
print("ACTUAL OUTPUT ===> ",df_test.dec_output.values[19])
print("="*50)
print("BEAM SEARCH OUTPUT ,  SCORE")
bm = (beam_search(df_test.enc_input.values[19],model,3))
for i in bm:
    print(i)

INPUT SENTENCE ===>  he went to nagano in school trip and ski there
ACTUAL OUTPUT ===>  he went to nagano once on a school trip and skied there <end>
BEAM SEARCH OUTPUT ,  SCORE
('he went to nagano in the school trip and skiing there <end>', -4.4937983)
('he went to nagano in school trip and skiing there <end>', -4.640549)
('he went to nagano in the school trip and skiing ski there <end>', -7.8885984)


In [36]:
print("INPUT SENTENCE ===> ",df_test.enc_input.values[50])
print("="*50)
print("ACTUAL OUTPUT ===> ",df_test.dec_output.values[50])
print("="*50)
print("BEAM SEARCH OUTPUT ,  SCORE")
bm = (beam_search(df_test.enc_input.values[50],model,3))
for i in bm:
    print(i)

INPUT SENTENCE ===>  yesterday i checked how to set the environment up for developing iphone applications using other is account
ACTUAL OUTPUT ===>  yesterday i checked how to set the environment up for developing iphone applications using another person is account <end>
BEAM SEARCH OUTPUT ,  SCORE
('yesterday i checked how to set the environment up for developing the iphone applications using other <end>', -4.739544)
('yesterday i checked how to set the environment up for developing the iphone applications using other is account <end>', -5.0009623)
('yesterday i checked how to set the environment up for developing the iphone applications using other thanksgiving <end>', -5.190677)
